In [ ]:
!pip install datasets
!pip install transformers
!pip install torchaudio
!pip install jiwer
!pip install soundfile
!pip install librosa
!pip install evaluate

In [ ]:
from transformers import pipeline
from datasets import load_dataset, load_metric, Audio, ClassLabel, load_from_disk, Features, Value
import evaluate

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# age_groups = ['sixties', 'seventies', 'eighties', 'nineties']
# # test = load_dataset('mozilla-foundation/common_voice_8_0', 'fy-NL', split='test')
# # test_set_13 = test.filter(lambda sample: age_groups.count(sample["age"]) == 1)

# test = load_dataset('mozilla-foundation/common_voice_8_0', 'fy-NL', split='test', use_auth_token=True)

In [ ]:
print("----------------- Loading Datasets... -----------------")
features = Features(
    {
        "client_id": Value("string"),
#         "path": Value("string"),
        "path": Audio(sampling_rate=48_000),
        "sentence": Value("string"),
        "up_votes": Value("int64"),
        "down_votes": Value("int64"),
        "age": Value("string"),
        "gender": Value("string"),
        "accents": Value("string"),
        "locale": Value("string"),
        "segment": Value("string"),
    }
)



# To prepare the csv:
# add column 'audio' with absolut path to the audio files (Excel function: =CONCAT('path/to/audio_files/folder' B2))
test_from_csv = load_dataset('csv', data_files={'test': 'path/to/test.csv', },
                                   data_dir="path/to/test/audio/folder")

test_from_csv = test_from_csv.cast(features)

test_from_csv = test_from_csv.remove_columns(
    ["accents", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])

test = test_from_csv['test']

test['path']
print("----------------- Loading Datasets complete. -----------------\n\n")

In [ ]:
import re
chars_to_remove_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\']'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_remove_regex, '', batch["sentence"]).lower()
    return batch

test = test.map(remove_special_characters)

In [ ]:
!git lfs

In [ ]:
import time

start = time.time()
transcriber = pipeline("automatic-speech-recognition", model='huggingface/your-repo')
wer = evaluate.load("wer")

labels = []
preds = []
i = 0

for recording in test:
    label = labels.append(recording['sentence'])
    pred = preds.append(transcriber(recording['path'])['text'])
    i += 1
    print(str(i) + '/' + str(len(test)))
    
print(wer.compute(predictions=preds, references=labels))
end = time.time()
print(end - start)